In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
import itertools

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array
import tarfile


# Extract LFW dataset if not already extracted
if not os.path.exists('lfw'):
    with tarfile.open('/content/lfw-funneled.tgz', 'r:gz') as tar:
        tar.extractall()

# Function to load images and labels from LFW dataset
def load_lfw_dataset(data_dir):
    images = []
    labels = []

    person_folders = sorted(os.listdir(data_dir))
    label_map = {person: idx for idx, person in enumerate(person_folders)}

    for person in person_folders:
        person_dir = os.path.join(data_dir, person)
        if os.path.isdir(person_dir):
            for image_file in os.listdir(person_dir):
                image_path = os.path.join(person_dir, image_file)
                image = load_img(image_path, color_mode='grayscale', target_size=(112, 92))
                image = img_to_array(image) / 255.0  # Normalize pixel values
                images.append(image)
                labels.append(label_map[person])

    return np.array(images), np.array(labels)

# Load LFW dataset
data_dir = 'lfw_funneled'
images, labels = load_lfw_dataset(data_dir)

# Split dataset into train, validation, and test sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# Print dataset shapes for verification
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_valid shape:', x_valid.shape)
print('y_valid shape:', y_valid.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)


x_train shape: (9527, 112, 92, 1)
y_train shape: (9527,)
x_valid shape: (1059, 112, 92, 1)
y_valid shape: (1059,)
x_test shape: (2647, 112, 92, 1)
y_test shape: (2647,)


In [ ]:
def create_pairs(x, y):
    pairs = []
    labels = []
    n_classes = np.max(y) + 1
    digit_indices = [np.where(y == i)[0] for i in range(n_classes)]

    for idx1 in range(len(x)):
        x1 = x[idx1]
        label1 = y[idx1]

        # Positive pair
        idx2 = np.random.choice(digit_indices[label1])
        x2 = x[idx2]
        pairs.append([x1, x2])
        labels.append(1)

        # Negative pair
        label2 = (label1 + np.random.randint(1, n_classes)) % n_classes
        if len(digit_indices[label2]) > 0:  # Check if there are samples for negative pair
            idx2 = np.random.choice(digit_indices[label2])
            x2 = x[idx2]
            pairs.append([x1, x2])
            labels.append(0)

    return np.array(pairs), np.array(labels)


def build_base_network(input_shape):
    # Define base CNN model to extract features from images
    model = Sequential([
        Conv2D(64, (10, 10), activation='relu', input_shape=input_shape),
        MaxPooling2D(),
        Conv2D(128, (7, 7), activation='relu'),
        MaxPooling2D(),
        Conv2D(128, (4, 4), activation='relu'),
        MaxPooling2D(),
        Conv2D(256, (4, 4), activation='relu'),
        Flatten(),
        Dense(4096, activation='sigmoid')
    ])
    return model

def euclidean_distance(vectors):
    # Custom Lambda layer to compute Euclidean distance between two embeddings
    x, y = vectors
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def euclidean_distance_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

# Build Siamese network model
im_rows, im_cols = 112, 92
input_shape = (im_rows, im_cols, 1)
base_network = build_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape=euclidean_distance_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
model.summary()

# Create pairs for training and validation
train_pairs, train_labels = create_pairs(x_train, y_train)
valid_pairs, valid_labels = create_pairs(x_valid, y_valid)



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 112, 92, 1)]         0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 112, 92, 1)]         0         []                            
                                                                                                  
 sequential_1 (Sequential)   (None, 4096)                 2636473   ['input_3[0][0]',             
                                                          6          'input_4[0][0]']             
                                                                                                  
 lambda_1 (Lambda)           (None, 1)                    0         ['sequential_1[0][0]',  

In [ ]:
# Train the model
history = model.fit(
    [train_pairs[:, 0], train_pairs[:, 1]], train_labels,
    batch_size=128,
    epochs=50,
    validation_data=([valid_pairs[:, 0], valid_pairs[:, 1]], valid_labels)
)



Epoch 1/50
134/134 [==============================] - 4462s 33s/step - loss: 2.6789 - accuracy: 0.4307 - val_loss: 5.4738 - val_accuracy: 0.1470
Epoch 2/50
134/134 [==============================] - 4375s 33s/step - loss: 2.6506 - accuracy: 0.4514 - val_loss: 5.4472 - val_accuracy: 0.1763
Epoch 3/50
  5/134 [>.............................] - ETA: 1:07:04 - loss: 2.3987 - accuracy: 0.4672

In [ ]:
# Evaluate the model on test set
test_pairs, test_labels = create_pairs(x_test, y_test)
score = model.evaluate([test_pairs[:, 0], test_pairs[:, 1]], test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Plot training history (accuracy and loss)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()



In [ ]:
# Optionally, save the trained model
model.save('siamese_network_lfw_model.h5')

# Load the trained model
loaded_model = load_model('siamese_network_lfw_model.h5', custom_objects={'euclidean_distance': euclidean_distance, 'euclidean_distance_output_shape': euclidean_distance_output_shape})

# Use the loaded model to make predictions
test_distance = loaded_model.predict([test_pairs[:, 0], test_pairs[:, 1]])
y_pred = (test_distance < 0.5).astype(int).reshape(-1)

# Calculate accuracy
accuracy = accuracy_score(test_labels, y_pred)
print("Loaded model accuracy:", accuracy)

# Confusion matrix and classification report
cnf_matrix = confusion_matrix(test_labels, y_pred)

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not Same', 'Same'], title='Confusion matrix, without normalization')
plt.show()

# Print classification report
print(classification_report(test_labels, y_pred))